In [105]:
import pandas as pd
from matplotlib import pyplot as plt
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from rutas import archivo_imagenes_etiquetadas_csv
from rutas import archivo_logs_csv
from rutas import archivo_perfiles_json

df=pd.read_csv(archivo_imagenes_etiquetadas_csv)
df_logs=pd.read_csv(archivo_logs_csv)

#Porcentajes según el tipo de imagen


- Se toma la columna 'mimetype' del dataFrame para obtener la extensión del tipo de imagen.
- Aplico el método split('x') para dividir la cadena y asi obtener la extensión.
- Realizo un conteo de cada extensión. Se generá una serie  con las extensiones  y sus respectiva   cantidad de ocurrencias.
- Almaceno en la variable etiquetas los índices de la serie(extensiones).



In [ ]:

extensiones = df['mimetype'].str.split('/').str[1].value_counts()
etiquetas = extensiones.index

# Generar el gráfico de torta
plt.pie(extensiones, labels=etiquetas, autopct='%1.1f%%',
        shadow=True, startangle=120, labeldistance=1.1)

plt.axis('equal')
plt.legend(etiquetas)
plt.title("Tipos  de imágenes")

plt.show()
df

#Alto y ancho máximos

- Se toma la columna 'resolucion' del DataFrame para obtener el alto y ancho de las imagenes.
- Aplico el método split('x') para dividir la cadena y asi obtener el alto y el ancho de cada imagen.
- Se hace la conversión a entero tanto del alto como del ancho y aplico la funcion max para obtener el máximo.

In [ ]:
maximo_alto = df['resolucion'].str.split('x').str[1].astype(int).max()
maximo_ancho = df['resolucion'].str.split('x').str[0].astype(int).max()

print(f"Máximo alto: {maximo_alto}px")
print(f"Máximo ancho: {maximo_ancho}px")

In [ ]:
#inciso 7, archivo imagenes clasificadas
#Tags mas utilizados

df_7=df.copy()
df_7['tags']=df_7['tags'].str.split(',')
df_7=df_7['tags'].explode().str.strip(' ').reset_index()
df_7=df_7.groupby('tags')['index'].count().reset_index(name="Usos")
df_7=df_7.rename(columns={'tags': 'Tags'})

print("Tags más utilizados")
df_7

In [109]:
#inciso 8, archivo imagenes clasificadas
#Imagenes promedio

df_etiq=pd.read_csv(archivo_imagenes_etiquetadas_csv)
df_etiq=df_etiq.rename(columns={'ultimo_perfil': 'Usuario'})

df_json=pd.read_json(archivo_perfiles_json)

outer_join=pd.merge(df_etiq,df_json,on="Usuario",how="outer")
tabla_final=outer_join.groupby(outer_join["Usuario"])["tamaño"].sum().reset_index(name="Tamaño (MB)")

print('Imagenes promedio')
tabla_final

Imagenes promedio


,Usuario,Tamaño (MB)
0,Mariana,0.85
1,Martin,1.06
2,Rogelio,0.00


In [ ]:
#inciso 1, archivo logs
#Operaciones por dias de la semana

from datetime import datetime
df_dias =pd.DataFrame({'Dia': [0, 1, 2, 3, 4, 5, 6]})

df_logs1=df_logs.copy()
df_logs1['Dia']=df_logs1['Fecha y Hora'].apply(lambda x: datetime.fromtimestamp(x))
df_logs1=df_logs1.groupby(df_logs1['Dia'].dt.day_of_week).size().reset_index(name="Cantidad")

outer_join=pd.merge(df_dias,df_logs1,on='Dia',how="outer")
outer_join=outer_join['Cantidad'].fillna(0).astype(int).reset_index()
outer_join['index']=['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']

colores = ['red', 'green', 'skyblue', 'pink', 'gray', 'yellow', 'brown']

plt.bar(outer_join["index"], outer_join["Cantidad"],color=colores)
plt.title('Operaciones por dias de la semana')
plt.xlabel('Dia de la semana')
plt.ylabel('Operaciones')
plt.show()

In [ ]:
#inciso 2, archivos logs
#Uso por genero

df_logs2=df_logs.copy().rename(columns={'Nick': 'Usuario'})
inner_join=pd.merge(df_logs2,df_json,on="Usuario",how="inner")
tabla_final=inner_join.groupby(inner_join["Genero"])["Genero"].count().reset_index(name="Cantidad")

plt.pie(tabla_final["Cantidad"], labels=tabla_final["Genero"], autopct='%1.1f%%',
        shadow=False, startangle=120, labeldistance=1.1)
plt.axis('equal')
plt.legend(tabla_final["Genero"])
plt.title("Uso de aplicación por género")

plt.show()